# Ford Fulkerson アルゴリズム(最大フロー問題)

1. 残余グラフ上の容量(cap)が0の辺を通らないStartからTerminalまでのパスを見つける
2. パスにおける容量(cap)の最小値をFとするとき、パス上に流量Fだけ流す。  ここで逆方向の変については、元々のグラフの流量をFだけ減らす
3. １でパスが見つからなくなるまで 1, 2 を繰り返す

### C++での実装
```cpp
struct Edge {
　　　　　// to: 行き先, cap: 容量, rev: 行き先から見た自分の番号
  int to, cap, rev;
};
class MaximumFlow {
 public:
  // 最初の変数はメンバー初期化リストの記法を用いて設定する
  explicit MaximumFlow(int n) : size(n), used(n + 1, false), G(n + 1) {}

  void add_edge(int a, int b, int c) {
    int current_Ga = G[a].size();  // 現時点でのG[a]の要素数
    int current_Gb = G[b].size();  // 現時点でのG[b]の要素数
    G[a].push_back(Edge({b, c, current_Gb}));
    G[b].push_back(Edge({a, 0, current_Ga}));
  }

  // F はスタートからposに到達する過程での残余グラフの辺の容量の最小値
  int dfs(int pos, int goal, int F) {
    if (pos == goal) return F;
    used[pos] = true;
    // 探索
    for (int i = 0; i < G[pos].size(); i++) {
      // 容量 0 の辺は扱えない
      if (G[pos][i].cap == 0) continue;

      // 既に訪問んした頂点にはいかない
      if (used[G[pos][i].to] == true) continue;

      // 目的地までのパスを探す
      int flow = dfs(G[pos][i].to, goal, min(F, G[pos][i].cap));

      if (flow >= 1) {
        G[pos][i].cap -= flow;
        G[G[pos][i].to][G[pos][i].rev].cap += flow;
        return flow;
      }
    }
    // 全ての辺を探索しても見つからなかった場合
    return 0;
  }

  int max_flow(int start, int goal) {
    int total_flow = 0;
    while (true) {
      for (int i = 0; i <= size; i++) used[i] = false;
      int F = dfs(start, goal, 1000000000);

      // 流せなくなったら終了
      if (F == 0) break;

      total_flow += F;
    }
    return total_flow;
  }

 private:
  int size;
  vector<bool> used;
  vector<vector<Edge>> G;
};
```



## 最大フロー問題の応用例 - 二部マッチング問題
二部グラフに辺の向きをつけ、スタート、ゴールをそれぞれ追加し、辺の容量を全て 1 とすると、二部マッチングの最大本数は、 s から t までの最大フロー問題に帰着する




### Pythonでの実装

In [1]:
import math
from sys import stdin, setrecursionlimit
from typing import List

setrecursionlimit(10**6)


# graphのadj要素をそれぞれ、{to: 行き先, cap: 辺の容量, rev: 逆のEdge
class Edge:
    def __init__(self, to, cap, rev: "Edge"):
        self.to = to
        self.cap = cap
        self.rev = rev


class FordFulkerson:
    def __init__(self, N):
        self.size = N
        self.graph: List[List[Edge]] = [[] for _ in range(N)]

    def add_edge(self, a, b, c):
        e = Edge(b, c, None)
        rev = Edge(a, 0, e)
        e.rev = rev
        self.graph[a].append(e)
        self.graph[b].append(rev)

    def dfs(self, pos, goal, F):
        # ゴールに到着
        if pos == goal:
            return F

        self.visited[pos] = True

        for e in self.graph[pos]:
            # 容量 0 の辺は使えない
            if e.cap == 0:
                continue
            # 既に訪問した頂点に行かない
            if self.visited[e.to]:
                continue
            # 目的地までのパスを探す
            flow = self.dfs(e.to, goal, min(F, e.cap))
            # フローを流せる場合、残余グラフの容量を flow だけ増減させる
            if flow:
                e.cap -= flow
                e.rev.cap += flow
                return flow
        # すべての辺を探索しても見つからなかった
        return 0

    def max_flow(self, s, t):
        ans = 0
        while True:
            self.visited = [False] * self.size
            F = self.dfs(s, t, math.inf)

            # フローを流せなくなったら操作終了
            if F == 0:
                break
            ans += F
        return ans


def main():
    N, M = map(int, stdin.readline().split())
    ff = FordFulkerson(N)
    for _ in range(M):
        A, B, C = map(int, stdin.readline().split())
        A, B = A-1, B-1
        ff.add_edge(A, B, C)

    print(ff.max_flow(0, N-1))